In [1]:
import os, sys, gc
import time
import glob
import pickle
import copy
import json
import random
from collections import OrderedDict, namedtuple
import multiprocessing
import threading
import traceback

from typing import Tuple, List

import h5py
from tqdm import tqdm, tqdm_notebook

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cv2
from PIL import Image


import torch
import torchvision
import torch.nn.functional as F

from torch import nn, optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR

import torchmetrics
import pl_bolts
import pytorch_lightning as pl



from IPython.display import display, clear_output

import faiss

from modules.AugsDS_v13 import *
from modules.eval_functions import *
from modules.eval_metrics import evaluate

sys.path.append('./modules')

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


 - Found: 861 screenshots.  SCREENSHOT_DIR=./FB_page_qry


In [2]:
if __name__ == '__main__':
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
from modules.Facebook_AFMultiGPU_model_v23_sy_v9 import ArgsT23_EffNetV2, FacebookModel

 - Found: 861 screenshots.  SCREENSHOT_DIR=./FB_page_qry
 - CFG A: arcface, adam, all_gpus_ddp


In [4]:
args = ArgsT23_EffNetV2()

args.pretrained_bb = False

args.arc_classnum = 40
print(args) 

 = = = = = = = = = = ArgList = = = = = = = = = =
ALL_FOLDERS                   : ['query_images', 'reference_images', 'training_images', 'imagenet_images', 'face_frames']
BACKBONE_GPUS                 : [0, 1, 2, 3]
BATCH_SIZE                    : 88
DATASET_WH                    : (384, 384)
DS_DIR                        : /home/spc-a6000/PycharmProjects/facebookproject/dataset_jpg_384x384
DS_INPUT_DIR                  : /home/spc-a6000/PycharmProjects/facebookproject/dataset
FrmFaces_SAMPLES              : ./FrameFaces_samples_v.pickle
GeM_opt_p                     : True
GeM_p                         : 3.0
ImgNet_SAMPLES                : ./ImageNet_samples_v.pickle
N_WORKERS                     : 10
OUTPUT_WH                     : (160, 160)
QRY_AUGS_INTENSITY            : 0.75
accelerator                   : ddp
arc_bottleneck                : None
arc_classnum                  : 40
arc_devices_v                 : None
arc_m                         : 0.4
arc_s                      

# Building model

In [5]:
model = FacebookModel(args)

 - Total weights: 54.45M


# Loading ckpt

In [6]:
ckpt_filename = './checkpoints/sjy_test10/epoch=9-step=42619_LIGHT.ckpt'
_ = model.restore_checkpoint(ckpt_filename)

 - Loading model state_dict.


 - WARNING: key="criterion.kernel" not found in saved checkpoint. Weights will not be loaded.


 - Restored checkpoint: ./checkpoints/sjy_test10/epoch=9-step=42619_LIGHT.ckpt.


# Inference configuration

In [7]:
do_simple_augmentation = False
K = 500

BATCH_SIZE   = 128
N_WORKERS    = 27
DS_INPUT_DIR = f'./all_datasets/dataset'
ALL_FOLDERS  = ['query_images', 'reference_images', 'training_images']

args.ALL_FOLDERS = ALL_FOLDERS
args.BATCH_SIZE = BATCH_SIZE
args.N_WORKERS = N_WORKERS
args.DS_INPUT_DIR = DS_INPUT_DIR

In [8]:
while DS_INPUT_DIR[-1] in ['/', r'\\']:
    DS_INPUT_DIR = DS_INPUT_DIR[:-1]
    
# Path where the rescaled images will be saved
args.DS_DIR = f'{args.DS_INPUT_DIR}_jpg_{args.DATASET_WH[0]}x{args.DATASET_WH[1]}'

# Data Source

In [9]:
if any( [not os.path.exists(os.path.join(args.DS_DIR, folder)) for folder in args.ALL_FOLDERS] ):
    assert os.path.exists(args.DS_INPUT_DIR), f'DS_INPUT_DIR not found: {args.DS_INPUT_DIR}'

    resize_dataset(
        ds_input_dir=args.DS_INPUT_DIR,
        ds_output_dir=args.DS_DIR,
        output_wh=args.DATASET_WH,
        output_ext='jpg',
        num_workers=args.N_WORKERS,
        ALL_FOLDERS=args.ALL_FOLDERS,
        verbose=False,
    )

print('Paths:')
print(' - DS_INPUT_DIR:', args.DS_INPUT_DIR)
print(' - DS_DIR:      ', args.DS_DIR)

assert os.path.exists(args.DS_DIR), f'DS_DIR not found: {args.DS_DIR}'

try:
    public_ground_truth_path = os.path.join(args.DS_DIR, 'public_ground_truth.csv')
    public_gt = pd.read_csv( public_ground_truth_path)

except:
    public_ground_truth_path = os.path.join(args.DS_INPUT_DIR, 'public_ground_truth.csv')
    public_gt = pd.read_csv( public_ground_truth_path)

Paths:
 - DS_INPUT_DIR: ./all_datasets/dataset
 - DS_DIR:       ./all_datasets/dataset_jpg_384x384


# Datasets

In [10]:
ds_qry_full = FacebookDataset(
    samples_id_v=[f'Q{i:05d}' for i in range(50_000)],
    do_augmentation=False,
    ds_dir=args.DS_DIR,
    output_wh=args.OUTPUT_WH,
    channel_first=True,
    norm_type= args.img_norm_type,
    verbose=True,
)
# ds_qry_full.plot_sample(4)


ds_ref_full = FacebookDataset(
    samples_id_v=[f'R{i:06d}' for i in range(1_000_000)],
    do_augmentation=False,
    ds_dir=args.DS_DIR,
    output_wh=args.OUTPUT_WH,
    channel_first=True,
    norm_type=args.img_norm_type,
    verbose=True,
)
# ds_ref_full.plot_sample(4)


ds_trn_full = FacebookDataset(
    samples_id_v=[f'T{i:06d}' for i in range(1_000_000)],
    do_augmentation=False,
    ds_dir=args.DS_DIR,
    output_wh=args.OUTPUT_WH,
    channel_first=True,
    norm_type=args.img_norm_type,
    verbose=True,
)
# ds_trn_full.plot_sample(4)



dl_qry_full = DataLoader(
        ds_qry_full,
        batch_size=args.BATCH_SIZE,
        num_workers=args.N_WORKERS,
        shuffle=False,
    )

dl_ref_full = DataLoader(
    ds_ref_full,
    batch_size=args.BATCH_SIZE,
    num_workers=args.N_WORKERS,
    shuffle=False,
)

dl_trn_full = DataLoader(
    ds_trn_full,
    batch_size=args.BATCH_SIZE,
    num_workers=args.N_WORKERS,
    shuffle=False,
)

### Query embeddings

In [11]:
embed_qry_d = calc_embed_d(
    model, 
    dataloader=dl_qry_full,
    do_simple_augmentation=do_simple_augmentation
)

 - Setting model's device: cuda:0


100%|██████████| 391/391 [01:39<00:00,  3.91it/s]


### Reference embeddings

In [12]:
aug = '_AUG' if do_simple_augmentation else ''
submission_path = ckpt_filename.replace('.ckpt', f'_{args.OUTPUT_WH[0]}x{args.OUTPUT_WH[1]}{aug}_REF.h5')
scores_path = submission_path.replace('.h5', '_match_d.pickle')

In [13]:
embed_ref_d = calc_embed_d(
    model, 
    dataloader=dl_ref_full, 
    do_simple_augmentation=do_simple_augmentation
)

save_submission(
    embed_qry_d,
    embed_ref_d,
    save_path=submission_path,
)

match_d = calc_match_scores(embed_qry_d, embed_ref_d, k=K)
save_obj(match_d, scores_path)

  0%|          | 0/7813 [00:00<?, ?it/s]

 - Setting model's device: cuda:0


100%|██████████| 7813/7813 [53:57<00:00,  2.41it/s]


 - Saved: ./checkpoints/sjy_test10/epoch=9-step=42619_LIGHT_160x160_REF.h5


100%|██████████| 100/100 [45:56<00:00, 27.57s/it]


Saved: ./checkpoints/sjy_test10/epoch=9-step=42619_LIGHT_160x160_REF_match_d.pickle


### Public GT validation

In [14]:
eval_d = evaluate(
    submission_path=submission_path,
    gt_path=public_ground_truth_path,
    is_matching=False,
)

 Running evaluation on: "./checkpoints/sjy_test10/epoch=9-step=42619_LIGHT_160x160_REF.h5"
{
  "average_precision": 0.6507371773869689,
  "recall_p90": 0.5301542776998598
}


### Training embeddings

In [15]:
aug = '_AUG' if do_simple_augmentation else ''
submission_path = ckpt_filename.replace('.ckpt', f'_{args.OUTPUT_WH[0]}x{args.OUTPUT_WH[1]}{aug}_TRN.h5')
scores_path = submission_path.replace('.h5', '_match_d.pickle')

In [16]:
embed_trn_d = calc_embed_d(
    model, 
    dataloader=dl_trn_full, 
    do_simple_augmentation=do_simple_augmentation
)

save_submission(
    embed_qry_d,
    embed_trn_d,
    save_path=submission_path,
)

  0%|          | 0/7813 [00:00<?, ?it/s]

 - Setting model's device: cuda:0


100%|██████████| 7813/7813 [31:03<00:00,  4.19it/s]


 - Saved: ./checkpoints/sjy_test10/epoch=9-step=42619_LIGHT_160x160_TRN.h5


In [17]:
match_d = calc_match_scores(embed_qry_d, embed_trn_d, k=K)
save_obj(match_d, scores_path)

100%|██████████| 100/100 [23:13<00:00, 13.93s/it]


Saved: ./checkpoints/sjy_test10/epoch=9-step=42619_LIGHT_160x160_TRN_match_d.pickle
